In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
import numpy as np
from tqdm import tqdm
from pathlib import Path
import openslide
import pandas as pd
import pickle

In [4]:
from fastai.callbacks.csv_logger import CSVLogger

In [5]:
from object_detection_fastai.helper.object_detection_helper import *
from object_detection_fastai.helper.wsi_loader import *
from object_detection_fastai.loss.RetinaNetFocalLoss import RetinaNetFocalLoss
from object_detection_fastai.models.RetinaNet import RetinaNet
from object_detection_fastai.callbacks.callbacks import BBLossMetrics, BBMetrics, PascalVOCMetric, PascalVOCMetricByDistance

In [6]:
slides_train = list([ "14_EIPH_568381 Turnbull blue.svs", 
                           "26_EIPH_566482 L Berliner Blau.svs",
                              "01_EIPH_563479 Berliner Blau.svs", 
                              "02_EIPH_574162 berliner blau.svs",
                               "28_EIPH_569948 L Turnbull blue.svs", 
                        ])

slides_val = list(set(["11_EIPH_575697 Berliner Blau.svs", 
                            "20_EIPH_576150 berliner blau.svs", 
                            "22_EIPH_575216 Turnbull blue.svs"]))

In [9]:
annotations_path = Path("../../Statistics/EIPH_Annotations.pkl")
annotations = pd.read_pickle(annotations_path)
annotations_train = annotations[annotations["image_name"].isin(slides_train)]
annotations_val = annotations[annotations["image_name"].isin(slides_val)]
annotations_train.head()

,id,image_id,image_set,species,image_name,image_type,grade,vector,unique_identifier,user_id,deleted,last_editor,data_set_name,version
105389,1417177,3526,245,equine,14_EIPH_568381 Turnbull blue.svs,TurnbullBlue,0,"{'x1': 17958, 'x2': 18042, 'y1': 4266, 'y2': 4...",b37370a6-d4c9-48e1-ad11-1ce32c895513,1,False,1,SREP,SREP_V2
105390,1417178,3526,245,equine,14_EIPH_568381 Turnbull blue.svs,TurnbullBlue,0,"{'x1': 17999, 'x2': 18113, 'y1': 4245, 'y2': 4...",01c4de6f-2641-4696-a971-cb62b0dca32e,1,False,1,SREP,SREP_V2
105391,1417188,3526,245,equine,14_EIPH_568381 Turnbull blue.svs,TurnbullBlue,0,"{'x1': 17715, 'x2': 17803, 'y1': 4550, 'y2': 4...",14d331ee-a150-48e2-a405-fea2dc4caca8,1,True,1,SREP,SREP
105392,1417189,3526,245,equine,14_EIPH_568381 Turnbull blue.svs,TurnbullBlue,0,"{'x1': 17780, 'x2': 17886, 'y1': 4793, 'y2': 4...",9049128f-0afe-4d66-8659-ef74a1dd911f,1,True,1,SREP,SREP
105393,1417197,3526,245,equine,14_EIPH_568381 Turnbull blue.svs,TurnbullBlue,0,"{'x1': 18905, 'x2': 19011, 'y1': 4702, 'y2': 4...",02747511-c950-40d9-b72a-d84ce8ede147,1,False,1,SREP,SREP_V2


In [10]:
slides_path = Path("../../../Slides")
files = {slide.name: slide for slide in slides_path.rglob("*.svs")  if slide.name in slides_train + slides_val}
files

{'01_EIPH_563479 Berliner Blau.svs': PosixPath('../../../Slides/Equine/01_EIPH_563479 Berliner Blau.svs'),
 '02_EIPH_574162 berliner blau.svs': PosixPath('../../../Slides/Equine/02_EIPH_574162 berliner blau.svs'),
 '11_EIPH_575697 Berliner Blau.svs': PosixPath('../../../Slides/Equine/11_EIPH_575697 Berliner Blau.svs'),
 '14_EIPH_568381 Turnbull blue.svs': PosixPath('../../../Slides/Equine/14_EIPH_568381 Turnbull blue.svs'),
 '20_EIPH_576150 berliner blau.svs': PosixPath('../../../Slides/Equine/20_EIPH_576150 berliner blau.svs'),
 '22_EIPH_575216 Turnbull blue.svs': PosixPath('../../../Slides/Equine/22_EIPH_575216 Turnbull blue.svs'),
 '26_EIPH_566482 L Berliner Blau.svs': PosixPath('../../../Slides/Equine/26_EIPH_566482 L Berliner Blau.svs'),
 '28_EIPH_569948 L Turnbull blue.svs': PosixPath('../../../Slides/Equine/28_EIPH_569948 L Turnbull blue.svs')}

In [11]:
tfms = get_transforms(do_flip=True,
                      flip_vert=True,
                      #max_rotate=90,
                      max_lighting=0.0,
                      max_zoom=1.,
                      max_warp=0.0,
                      p_affine=0.5,
                      p_lighting=0.0,
                      #xtra_tfms=xtra_tfms,
                     )
tfms

([RandTransform(tfm=TfmCrop (crop_pad), kwargs={'row_pct': (0, 1), 'col_pct': (0, 1), 'padding_mode': 'reflection'}, p=1.0, resolved={}, do_run=True, is_random=True),
  RandTransform(tfm=TfmAffine (dihedral_affine), kwargs={}, p=1.0, resolved={}, do_run=True, is_random=True),
  RandTransform(tfm=TfmAffine (rotate), kwargs={'degrees': (-10.0, 10.0)}, p=0.5, resolved={}, do_run=True, is_random=True)],
 [RandTransform(tfm=TfmCrop (crop_pad), kwargs={}, p=1.0, resolved={}, do_run=True, is_random=True)])

In [12]:
size = 1024 
level = 0
bs = 16
train_images = 2500
val_images = 1500

In [13]:
def get_y_func(x):
    return x.y

In [14]:
anchors = create_anchors(sizes=[(32,32)], ratios=[1], scales=[0.6, 0.7, 0.9, 1.25, 1.5])

In [17]:
for i in range(len(slides_train)):
    
    torch.cuda.empty_cache()

    train_files = []
    val_files = []
    
    experiment_name = "EquineVsEquine-Ablation_{}".format(i)

    for image_name in slides_train[:i+1]:

        annotations = annotations_train[annotations_train["image_name"] == image_name]
        annotations = annotations[annotations["deleted"] == False]

        slide_path = files[image_name]
        labels =  list(annotations["grade"])
        bboxes = [[vector["x1"], vector["y1"], vector["x2"], vector["y2"]] for vector in annotations["vector"]]

        for grade in [0, 1, 2, 3, 4]:
            if grade not in set(labels):
                bboxes.append([0,0,0,0])
                labels.append(grade)
        
        train_files.append(SlideContainer(slide_path, y=[bboxes, labels],  level=level, width=size, height=size))

    for image_name in annotations_val["image_name"].unique():

        annotations = annotations_val[annotations_val["image_name"] == image_name]
        annotations = annotations[annotations["deleted"] == False]

        slide_path = files[image_name]
        labels =  list(annotations["grade"])
        bboxes = [[vector["x1"], vector["y1"], vector["x2"], vector["y2"]] for vector in annotations["vector"]]

        for grade in [0, 1, 2, 3, 4]:
            if grade not in set(labels):
                bboxes.append([0,0,0,0])
                labels.append(grade)
        
        val_files.append(SlideContainer(slide_path, y=[bboxes, labels],  level=level, width=size, height=size))
    
    train_files = list(np.random.choice(train_files, train_images))
    valid_files = list(np.random.choice(val_files, val_images))
    
    train =  ObjectItemListSlide(train_files, path=slides_path)
    valid = ObjectItemListSlide(valid_files, path=slides_path)
    item_list = ItemLists(slides_path, train, valid)
    lls = item_list.label_from_func(get_y_func, label_cls=SlideObjectCategoryList) #
    lls = lls.transform(tfms, tfm_y=True, size=size)
    data = lls.databunch(bs=bs, collate_fn=bb_pad_collate).normalize()
    
    crit = RetinaNetFocalLoss(anchors)
    encoder = create_body(models.resnet18, True, -2)
    model = RetinaNet(encoder, n_classes=data.train_ds.c, n_anchors=5, sizes=[32], chs=128, final_bias=-4., n_conv=3)
    
    voc = PascalVOCMetricByDistance(anchors, size, [str(i) for i in data.train_ds.y.classes[1:]], radius=25)
    learn = Learner(data, model, loss_func=crit, callback_fns=[BBMetrics, partial(CSVLogger, append=True, filename=experiment_name)], #BBMetrics, ShowGraph
                    metrics=[voc])

    learn.split([model.encoder[6], model.c5top5])
    learn.freeze_to(-2)
    
    learn.fit_one_cycle(3, 1e-3)
    
    learn.unfreeze()
    learn.fit_one_cycle(10, 1e-3)
    learn.fit_one_cycle(10, 1e-3)
        
    learn.destroy() 

epoch,train_loss,valid_loss,pascal_voc_metric_by_distance,BBloss,focal_loss,AP-0,AP-1,AP-2,AP-3,AP-4,time
0,0.558638,0.497701,0.357202,0.123201,0.374500,0.491639,0.446620,0.181877,0.392648,0.273227,04:20
1,0.308062,0.354262,0.558962,0.087355,0.266906,0.574851,0.568633,0.543985,0.542948,0.564394,04:19
2,0.225372,0.315765,0.593335,0.073498,0.242267,0.593054,0.634930,0.582199,0.541796,0.614696,04:16


epoch,train_loss,valid_loss,pascal_voc_metric_by_distance,BBloss,focal_loss,AP-0,AP-1,AP-2,AP-3,AP-4,time
0,0.229242,0.383997,0.484618,0.093213,0.290784,0.552084,0.646535,0.513121,0.350045,0.361304,04:24
1,0.293759,0.419276,0.471645,0.097931,0.321346,0.505179,0.424534,0.527451,0.558731,0.342332,04:16
2,0.300755,0.366624,0.590176,0.092464,0.274159,0.557456,0.711957,0.544631,0.490378,0.646460,04:23
3,0.286122,0.419268,0.591581,0.087809,0.331459,0.503951,0.478661,0.643045,0.578302,0.753944,04:17
4,0.246014,0.441942,0.551053,0.101223,0.340718,0.495589,0.575493,0.642707,0.474231,0.567246,04:14
5,0.226796,0.446755,0.588818,0.080708,0.366047,0.411971,0.526798,0.683740,0.530346,0.791234,04:13
6,0.190965,0.331153,0.614978,0.066770,0.264383,0.627159,0.592843,0.693478,0.551198,0.610213,04:17
7,0.161005,0.344305,0.641593,0.065689,0.278616,0.569772,0.693099,0.630296,0.620456,0.694345,04:14
8,0.140325,0.368927,0.624972,0.066880,0.302048,0.561159,0.666908,0.630921,0.565184,0.700691,04:16
9,0.135389,0.349300,0.598305,0.068526,0.280774,0.586722,0.686154,0.628673,0.559424,0.530553,04:14


epoch,train_loss,valid_loss,pascal_voc_metric_by_distance,BBloss,focal_loss,AP-0,AP-1,AP-2,AP-3,AP-4,time
0,0.132243,0.387540,0.588637,0.068926,0.318614,0.575561,0.650718,0.608148,0.558570,0.550190,04:12
1,0.182819,0.431025,0.539119,0.088484,0.342542,0.516474,0.727854,0.427364,0.555575,0.468329,04:26
2,0.211489,0.441476,0.542929,0.085982,0.355494,0.506955,0.539452,0.532349,0.568723,0.567168,04:15
3,0.205020,0.397850,0.568290,0.080457,0.317393,0.567836,0.550611,0.523699,0.626519,0.572784,04:15
4,0.184010,0.360103,0.586690,0.075752,0.284350,0.568889,0.613511,0.603416,0.493867,0.653768,04:19
5,0.174052,0.415727,0.565345,0.073252,0.342476,0.565780,0.613798,0.598787,0.495147,0.553211,04:15
6,0.144830,0.440475,0.597505,0.080298,0.360177,0.522869,0.601990,0.598715,0.649599,0.614351,04:13
7,0.116065,0.432285,0.605558,0.070306,0.361979,0.572169,0.680908,0.645860,0.489479,0.639372,04:13
8,0.097628,0.459800,0.599914,0.069087,0.390713,0.552647,0.685704,0.617119,0.540280,0.603822,04:13
9,0.092340,0.445359,0.608459,0.068434,0.376925,0.551561,0.686494,0.627065,0.567638,0.609535,04:14


this Learner object self-destroyed - it still exists, but no longer usable


epoch,train_loss,valid_loss,pascal_voc_metric_by_distance,BBloss,focal_loss,AP-0,AP-1,AP-2,AP-3,AP-4,time
0,0.681211,0.518188,0.462100,0.108603,0.409585,0.627985,0.281610,0.424380,0.524440,0.452085,04:05
1,0.342119,0.310587,0.580814,0.065839,0.244748,0.608630,0.698236,0.609021,0.376141,0.612041,04:19
2,0.269604,0.284454,0.519919,0.062904,0.221550,0.633721,0.655103,0.594380,0.324799,0.391593,04:16


...

this Learner object self-destroyed - it still exists, but no longer usable


epoch,train_loss,valid_loss,pascal_voc_metric_by_distance,BBloss,focal_loss,AP-0,AP-1,AP-2,AP-3,AP-4,time
0,0.573220,0.450881,0.484106,0.106950,0.343932,0.544406,0.589008,0.417452,0.266853,0.602809,04:29
1,0.339379,0.315734,0.548970,0.068862,0.246872,0.657077,0.656555,0.601996,0.287668,0.541555,04:10
2,0.263346,0.277799,0.531081,0.062048,0.215752,0.635540,0.740580,0.556569,0.274091,0.448624,04:18


epoch,train_loss,valid_loss,pascal_voc_metric_by_distance,BBloss,focal_loss,AP-0,AP-1,AP-2,AP-3,AP-4,time
0,0.277601,0.303638,0.548433,0.069742,0.233895,0.504920,0.741786,0.521917,0.471430,0.502111,04:17
1,0.328192,0.403663,0.446142,0.101658,0.302005,0.585511,0.582920,0.520199,0.210257,0.331824,04:29
2,0.340897,0.578861,0.402432,0.106724,0.472138,0.507171,0.473527,0.555122,0.207645,0.268694,04:07
3,0.329854,0.405162,0.398668,0.089884,0.315278,0.463877,0.709891,0.274773,0.224534,0.320264,04:46
4,0.292561,0.292956,0.548609,0.065269,0.227687,0.658583,0.719365,0.620322,0.431566,0.313208,04:22
5,0.261595,0.297578,0.649439,0.073871,0.223706,0.664296,0.576766,0.681908,0.544776,0.779450,04:16
6,0.240106,0.247817,0.607217,0.055523,0.192293,0.729208,0.706151,0.659059,0.490335,0.451335,04:20
7,0.222824,0.256478,0.578579,0.053950,0.202528,0.692256,0.721873,0.506184,0.459826,0.512755,04:18
8,0.203900,0.231755,0.621787,0.051865,0.179890,0.724928,0.779813,0.618973,0.389693,0.595525,04:16
9,0.197987,0.242374,0.581103,0.051986,0.190388,0.716588,0.781139,0.574432,0.318642,0.514714,04:20


epoch,train_loss,valid_loss,pascal_voc_metric_by_distance,BBloss,focal_loss,AP-0,AP-1,AP-2,AP-3,AP-4,time
0,0.204188,0.257749,0.601091,0.057691,0.200059,0.720607,0.762426,0.471654,0.395042,0.655725,04:17
1,0.235794,0.271164,0.629880,0.060836,0.210328,0.674346,0.671663,0.653740,0.509686,0.639966,04:16
2,0.281474,0.372811,0.454184,0.092999,0.279812,0.554704,0.688751,0.545965,0.166934,0.314565,04:15
3,0.266858,0.303409,0.623710,0.062867,0.240542,0.669588,0.712072,0.462458,0.565694,0.708736,04:17
4,0.250729,0.268731,0.659030,0.061586,0.207145,0.706162,0.773237,0.609039,0.524588,0.682123,04:19
5,0.233133,0.277415,0.591885,0.057358,0.220057,0.688494,0.741699,0.600018,0.312401,0.616816,04:19
6,0.215388,0.254216,0.538358,0.051578,0.202638,0.711267,0.753563,0.488278,0.324754,0.413927,04:20
7,0.203176,0.250189,0.536307,0.049331,0.200857,0.709381,0.774002,0.469001,0.312038,0.417112,04:17
8,0.188068,0.238828,0.581311,0.050865,0.187963,0.731554,0.767128,0.589783,0.349849,0.468240,04:17
9,0.183723,0.235977,0.583742,0.049979,0.185998,0.728805,0.774079,0.596525,0.339196,0.480103,04:20


this Learner object self-destroyed - it still exists, but no longer usable
